In [2]:
import os
os.environ["http_proxy"] = "http://127.0.0.1:8889"
os.environ["https_proxy"] = "http://127.0.0.1:8889"

## Step-1 Import

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

/home/dean/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step-2 Load dataset

In [4]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="../Introduction/datasets/ChnSentiCorp_htl_all.csv", split="train[:100%]")
dataset = dataset.filter(lambda example: example['review'] is not None)
dataset

Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

## Step-3 Split dataset

In [5]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

## Step-4 Data preprocessing

In [6]:
import torch
# hfl/chinese-macbert-large, 330M
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-large")

def process_function(examples):
    tokenized_example = tokenizer(examples["review"], truncation=True, max_length=32, padding="max_length")
    tokenized_example["label"] = examples["label"]
    return tokenized_example

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

Map: 100%|██████████| 777/777 [00:00<00:00, 22324.49 examples/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 777
    })
})

## Step-5 Create Model

In [7]:
from torch.optim import AdamW 

model = AutoModelForSequenceClassification.from_pretrained("hfl/chinese-macbert-large")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-macbert-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step-6 Create evaluate function 

In [8]:
import evaluate

acc_metrics = evaluate.load("accuracy")
f1_metrics = evaluate.load("f1")

In [9]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metrics.compute(predictions=predictions, references=labels)
    f1 = f1_metrics.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

## Step-7 Create TrainingArguments

In [10]:
train_args = TrainingArguments(
    output_dir="./checkpoints",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=32,
    gradient_checkpointing=True,
    optim="adafactor",
    per_device_eval_batch_size=32,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=1,
    metric_for_best_model="f1",
    load_best_model_at_end=True,
)
# train_args

## Step-8 Create Trainer

In [11]:
from transformers import DataCollatorWithPadding

for name, param in model.bert.named_parameters():
    param.requires_grad = False

trainer = Trainer(model=model,
                  args=train_args,
                  train_dataset=tokenized_datasets["train"],
                  eval_dataset=tokenized_datasets["test"],
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric,
                  )
                  

## Step-9 Train


In [1]:
trainer.train()

NameError: name 'trainer' is not defined

| Optimization | Target | GPU mem | Train time |
|--------------|--------|---------|------------|
| Baseline     |  -     |15.2 G     |  64s    |
| + Gradient Accumulation (BS 1, GA 32) | Forward activation value    |7 .4 G     |  259s     |
| + Gradient Checkpoints (BS 1, GA 32) | Forward activation value    | 7.2 G     |  422s     |
| + Adafactor Optimizer(BS 1, GA 32) | Optimizer state    |5.0 G     | 406s     |
| + Freeze model (BS 1, GA 32) | Forward activation value / gradient    | 3.5 G     |  178s     |
| + Data Length (BS 1, GA 32, MaxLength 32) | Forward activation value | 3.4 G     |  126s     |

## Step-10 Model evaluation

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

## Step-11 Model prediction

In [ ]:
trainer.predict(tokenized_datasets["test"])

## Step-12 Save and load

In [ ]:
model.save_pretrained("./save_models/rbt3")

model = AutoModelForSequenceClassification.from_pretrained("./save_models/rbt3")